# Spark Session

All distributed operations with Spark are done using so-called Spark Session. Usually one is already created by your cluster's administrator.

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
pip install kaggle

Python interpreter will be restarted.
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=df323c7f1b79dd22fbf20d489ab4794e716f942d2b8a56ed85591250dfcbea9e
  Stored in directory: /root/.cache/pip/wheels/ac/b2/c3/fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle
Python interpreter will be restarted.


In [0]:
dbutils.fs.rm("file:///root/.kaggle")

Out[1]: False

In [0]:
dbutils.fs.mkdirs("file:///root/.kaggle/")    #Create a new folder

Out[2]: True

In [0]:
dbutils.fs.cp("dbfs:///FileStore/kaggle.json","file:///root/.kaggle/") #copy kaggle.json in root/..

Out[3]: True

In [0]:
dbutils.fs.ls("file:///root/.kaggle/") #verify

Out[4]: [FileInfo(path='file:/root/.kaggle/kaggle.json', name='kaggle.json', size=66, modificationTime=1673528877003)]

In [0]:
!pip install kaggle
import kaggle

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9d0a0ff3-7e85-4fc8-910e-859d3ab5516c/bin/python -m pip install --upgrade pip' command.


In [0]:
!kaggle datasets download -d arashnic/covid19-hospital-treatment

  0%|                                               | 0.00/4.48M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 4.48M/4.48M [00:00<00:00, 59.1MB/s]


In [0]:
!unzip covid19-hospital-treatment.zip

Archive:  covid19-hospital-treatment.zip
  inflating: host_train.csv          


In [0]:
import os
file_name = "host_train.csv"
file_path = os.path.join(os.getcwd(), file_name)

In [0]:
dbutils.fs.cp(f"file://{file_path}","dbfs:///")
raw_lines = spark.read.csv(
    f"dbfs:///{file_name}"
)
#.to_pandas_on_spark()

Here all the libraries I'll need

In [0]:
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType
# for categorical variables you can do one-hot encoding
from pyspark.ml.feature import OneHotEncoder, StringIndexer
# how to do scaling and prepare number columns for feeding a model
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
# feature engineering
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.regression import LinearRegression

from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.feature import StandardScaler

from pyspark.sql.functions import col,isnan,when,count

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.classification import DecisionTreeClassifier

In [0]:
raw_lines.head()

Out[11]: Row(_c0='case_id', _c1='Hospital', _c2='Hospital_type', _c3='Hospital_city', _c4='Hospital_region', _c5='Available_Extra_Rooms_in_Hospital', _c6='Department', _c7='Ward_Type', _c8='Ward_Facility', _c9='Bed_Grade', _c10='patientid', _c11='City_Code_Patient', _c12='Type of Admission', _c13='Illness_Severity', _c14='Patient_Visitors', _c15='Age', _c16='Admission_Deposit', _c17='Stay_Days')

##########               HERE STARTS THE PROJECT                    ###############################

In [0]:
import pandas as pd
pandas_df = raw_lines.toPandas()
for i in raw_lines.columns:
    raw_lines = raw_lines.withColumnRenamed(i,pandas_df[i][0])

#pd.dataFrame(raw_lines).head(2)

In [0]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318439 entries, 0 to 318438
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   _c0     318439 non-null  object
 1   _c1     318439 non-null  object
 2   _c2     318439 non-null  object
 3   _c3     318439 non-null  object
 4   _c4     318439 non-null  object
 5   _c5     318439 non-null  object
 6   _c6     318439 non-null  object
 7   _c7     318439 non-null  object
 8   _c8     318439 non-null  object
 9   _c9     318326 non-null  object
 10  _c10    318439 non-null  object
 11  _c11    313907 non-null  object
 12  _c12    318439 non-null  object
 13  _c13    318439 non-null  object
 14  _c14    318439 non-null  object
 15  _c15    318439 non-null  object
 16  _c16    318439 non-null  object
 17  _c17    318439 non-null  object
dtypes: object(18)
memory usage: 43.7+ MB


In [0]:
raw_lines.show

Out[14]: <bound method DataFrame.show of DataFrame[case_id: string, Hospital: string, Hospital_type: string, Hospital_city: string, Hospital_region: string, Available_Extra_Rooms_in_Hospital: string, Department: string, Ward_Type: string, Ward_Facility: string, Bed_Grade: string, patientid: string, City_Code_Patient: string, Type of Admission: string, Illness_Severity: string, Patient_Visitors: string, Age: string, Admission_Deposit: string, Stay_Days: string]>

####Covid 19 treatments

The COVID-19 pandemic has placed an unprecedented strain on health systems in all the world, with rapidly increasing demand for healthcare in hospitals and intensive care units. As the pandemic escalates, determining the resulting needs for healthcare resources (beds, staff, equipment, intensive therapies) has become a key priority for many countries, principally in Italy where the pandemic hited the first in Europe, and puted our healthcare system in big difficulties. Projecting future demand requires estimates of how long patients with COVID-19 need different levels of hospital care. While healthcare management has various use cases for using data science, patient length of stay is one critical parameter to observe and predict for us if we want to improve the efficiency of the healthcare management in a hospitals.

The task is to accurately predict the Length of Stay for each patient on case by case basis so that the Hospitals can use this information for optimal resource allocation and better functioning. The length of stay is divided into 11 different classes ranging from 0-10 days to more than 100 days.

In [0]:
df_covid = raw_lines

In [0]:
df_covid.show(7) #To see the first 10 rows of the dataset

+-------+--------+-------------+-------------+---------------+---------------------------------+------------+---------+-------------+---------+---------+-----------------+-----------------+----------------+----------------+-----+-----------------+---------+
|case_id|Hospital|Hospital_type|Hospital_city|Hospital_region|Available_Extra_Rooms_in_Hospital|  Department|Ward_Type|Ward_Facility|Bed_Grade|patientid|City_Code_Patient|Type of Admission|Illness_Severity|Patient_Visitors|  Age|Admission_Deposit|Stay_Days|
+-------+--------+-------------+-------------+---------------+---------------------------------+------------+---------+-------------+---------+---------+-----------------+-----------------+----------------+----------------+-----+-----------------+---------+
|case_id|Hospital|Hospital_type|Hospital_city|Hospital_region|             Available_Extra_R...|  Department|Ward_Type|Ward_Facility|Bed_Grade|patientid|City_Code_Patient|Type of Admission|Illness_Severity|Patient_Visitors|  A

In [0]:
df_covid.dtypes ## all the columns are type string

Out[17]: [('case_id', 'string'),
 ('Hospital', 'string'),
 ('Hospital_type', 'string'),
 ('Hospital_city', 'string'),
 ('Hospital_region', 'string'),
 ('Available_Extra_Rooms_in_Hospital', 'string'),
 ('Department', 'string'),
 ('Ward_Type', 'string'),
 ('Ward_Facility', 'string'),
 ('Bed_Grade', 'string'),
 ('patientid', 'string'),
 ('City_Code_Patient', 'string'),
 ('Type of Admission', 'string'),
 ('Illness_Severity', 'string'),
 ('Patient_Visitors', 'string'),
 ('Age', 'string'),
 ('Admission_Deposit', 'string'),
 ('Stay_Days', 'string')]

In [0]:
df_covid.summary().toPandas()

,summary,case_id,Hospital,Hospital_type,Hospital_city,Hospital_region,Available_Extra_Rooms_in_Hospital,Department,Ward_Type,Ward_Facility,Bed_Grade,patientid,City_Code_Patient,Type of Admission,Illness_Severity,Patient_Visitors,Age,Admission_Deposit,Stay_Days
0,count,318439,318439,318439,318439,318439,318439,318439,318439,318439,318326,318439,313907,318439,318439,318439,318439,318439,318439
1,mean,159219.5,18.318840716246175,1.2568412061374585,4.771716943329627,0.7780980913081982,3.1976271676119055,None,None,None,2.625806958297338,65747.57947229916,7.251858836721821,None,None,3.2840992595104854,None,4880.749392346391,None
2,stddev,91925.276847557,8.633754885795044,1.535712038183557,3.1025353546703163,0.7525271792894246,1.1681714278597128,None,None,None,0.8731458045921738,37979.93644002388,4.745265561141453,None,None,1.7640613895592003,None,1086.7762536416237,None
3,min,1,1,0,1,0,0,Department,P,A,1.0,1,1.0,Emergency,Extreme,0,0-10,10010.0,0-10
4,25%,79615.0,11.0,0.0,2.0,0.0,2.0,None,None,None,2.0,32844.0,4.0,None,None,2.0,None,4186.0,None
5,50%,159198.0,19.0,1.0,5.0,1.0,3.0,None,None,None,3.0,65719.0,8.0,None,None,3.0,None,4741.0,None
6,75%,238826.0,26.0,2.0,7.0,1.0,4.0,None,None,None,3.0,98463.0,8.0,None,None,4.0,None,5408.0,None
7,max,case_id,Hospital,Hospital_type,Hospital_city,Hospital_region,Available_Extra_Rooms_in_Hospital,surgery,Ward_Type,Ward_Facility,Bed_Grade,patientid,City_Code_Patient,Urgent,Moderate,Patient_Visitors,Age,Admission_Deposit,Stay_Days


In [0]:
df_covid.count()  #the total number of rows in the  dataset  #318439

Out[19]: 318439

In [0]:
df_covid.describe().show()

+-------+---------------+------------------+------------------+------------------+------------------+---------------------------------+----------+---------+-------------+------------------+-----------------+-----------------+-----------------+----------------+------------------+------+------------------+---------+
|summary|        case_id|          Hospital|     Hospital_type|     Hospital_city|   Hospital_region|Available_Extra_Rooms_in_Hospital|Department|Ward_Type|Ward_Facility|         Bed_Grade|        patientid|City_Code_Patient|Type of Admission|Illness_Severity|  Patient_Visitors|   Age| Admission_Deposit|Stay_Days|
+-------+---------------+------------------+------------------+------------------+------------------+---------------------------------+----------+---------+-------------+------------------+-----------------+-----------------+-----------------+----------------+------------------+------+------------------+---------+
|  count|         318439|            318439|        

Convert the three variables in int ("Admission_Deposit", "Patient_Visitors", "Available_Extra_Rooms_in_Hospital")

In [0]:
df_covid = df_covid.withColumn("Admission_Deposit", df_covid.Admission_Deposit.cast('int'))

In [0]:
df_covid = df_covid.withColumn("Patient_Visitors", df_covid.Patient_Visitors.cast('int'))

In [0]:
df_covid = df_covid.withColumn("Available_Extra_Rooms_in_Hospital", df_covid.Available_Extra_Rooms_in_Hospital.cast('int'))

In [0]:
df_covid = df_covid.withColumn("Available_Extra_Rooms_in_Hospital",df_covid.Available_Extra_Rooms_in_Hospital.cast('int'))

I want to see the new dataset with the three new column in int and I'll make a new copy of the dataset

In [0]:
df_covid.show
df_new_covid = df_covid

I'll drop the columns that dosen't give me information in the previsions at the end.

Data Preprocessing

In [0]:
#None of these columns gives me important information, so I can delete them
df_new_covid = df_new_covid.drop("case_id")
df_new_covid = df_new_covid.drop("patientid")
df_new_covid = df_new_covid.drop("Hospital_region")
df_new_covid.show

Out[25]: <bound method DataFrame.show of DataFrame[Hospital: string, Hospital_type: string, Hospital_city: string, Available_Extra_Rooms_in_Hospital: int, Department: string, Ward_Type: string, Ward_Facility: string, Bed_Grade: string, City_Code_Patient: string, Type of Admission: string, Illness_Severity: string, Patient_Visitors: int, Age: string, Admission_Deposit: int, Stay_Days: string]>

In [0]:
df_new_covid

Out[26]: DataFrame[Hospital: string, Hospital_type: string, Hospital_city: string, Available_Extra_Rooms_in_Hospital: int, Department: string, Ward_Type: string, Ward_Facility: string, Bed_Grade: string, City_Code_Patient: string, Type of Admission: string, Illness_Severity: string, Patient_Visitors: int, Age: string, Admission_Deposit: int, Stay_Days: string]

In [0]:
df_new_covid.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_new_covid.columns]).show()
#which columns have missing values and how much, so in case I've columns with too much missing values I, like more than 60% of missing values, I can eleminate directly
#but like I see, the number off missing values is not too high, so I decide to mantain them all.

+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|Hospital|Hospital_type|Hospital_city|Available_Extra_Rooms_in_Hospital|Department|Ward_Type|Ward_Facility|Bed_Grade|City_Code_Patient|Type of Admission|Illness_Severity|Patient_Visitors|Age|Admission_Deposit|Stay_Days|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|       0|            0|            0|                                1|         0|        0|            0|      113|             4532|                0|               0|               1|  0|                1|        0|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+---

In [0]:

import numpy as np
df_new_covid.dtypes[3][1] == 'int'
range(0,len(df_new_covid.columns))
pandas_new = df_new_covid.toPandas()
pandas_new = pandas_new.iloc[1: , :]


In [0]:
#for the categorical data I'm filling the missing values with 0, for the numerical I'm filling with the median
for i in range(0,len(df_new_covid.columns)):
    if df_new_covid.dtypes[i][1] == 'int':
        df_new_covid = df_new_covid.fillna(value=np.median(pandas_new.iloc[:,i]))
    else:
        df_new_covid = df_new_covid.fillna(value="0")

In [0]:
#After filling the null values, here I control if I've anymore null values, to be sure
df_new_covid.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_new_covid.columns]).show()

+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|Hospital|Hospital_type|Hospital_city|Available_Extra_Rooms_in_Hospital|Department|Ward_Type|Ward_Facility|Bed_Grade|City_Code_Patient|Type of Admission|Illness_Severity|Patient_Visitors|Age|Admission_Deposit|Stay_Days|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|       0|            0|            0|                                0|         0|        0|            0|        0|                0|                0|               0|               0|  0|                0|        0|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+---

I've filled the null values in the numerical with the median and in categorical with 0

I'll create two pipelines, one for the categorical columns, and one for the numerical:

In [0]:
pipeline_categorical = Pipeline(stages=[
    StringIndexer(inputCol="Department", outputCol="DepartmentI"),
    OneHotEncoder(inputCol="DepartmentI", outputCol="DepartmentOhe"),
    
    StringIndexer(inputCol="Ward_Type", outputCol="Ward_TypeI"),
    OneHotEncoder(inputCol="Ward_TypeI", outputCol="Ward_TypeOhe"),
    
    StringIndexer(inputCol="Hospital_city", outputCol="Hospital_cityI"),
    OneHotEncoder(inputCol="Hospital_cityI", outputCol="Hospital_cityOhe"),
    
   StringIndexer(inputCol="City_Code_Patient", outputCol="City_Code_PatientI"),
    OneHotEncoder(inputCol="City_Code_PatientI", outputCol="City_Code_PatientOhe"),
    
    StringIndexer(inputCol="Ward_Facility", outputCol="Ward_FacilityI"),
    OneHotEncoder(inputCol="Ward_FacilityI", outputCol="Ward_FacilityOhe"),
    
    StringIndexer(inputCol="Hospital", outputCol="HospitalI"),
    
    StringIndexer(inputCol="Hospital_type", outputCol="Hospital_typeI"),
    
    
    StringIndexer(inputCol="Illness_Severity",outputCol="Illness_SeverityI"),  
    OneHotEncoder(inputCol="Illness_SeverityI", outputCol="Illness_SeverityOhe"),
    
    StringIndexer(inputCol="Bed_Grade",outputCol="Bed_GradeI"),  
    OneHotEncoder(inputCol="Bed_GradeI", outputCol="Bed_GradeOhe"),
    
    StringIndexer(inputCol="Age",outputCol="AgeI"),  
    OneHotEncoder(inputCol="AgeI", outputCol="AgeOhe"),
    
    StringIndexer(inputCol="Type of Admission",outputCol="Type of AdmissionI"),  
    OneHotEncoder(inputCol="Type of AdmissionI", outputCol="Type of AdmissionOhe"),
    
    VectorAssembler(
    inputCols=["City_Code_PatientOhe","Hospital_cityOhe","Type of AdmissionOhe","AgeOhe","DepartmentOhe","Ward_TypeOhe","Ward_FacilityOhe","HospitalI","Hospital_typeI","Illness_SeverityOhe","Bed_GradeOhe"],
  
    outputCol="features_categorical")
    
])

In [0]:
#For the numerical
pipeline_numerical = Pipeline(stages=[
    VectorAssembler(
        inputCols=["Available_Extra_Rooms_in_Hospital","Patient_Visitors","Admission_Deposit"],
        outputCol="features_numerical"
    ),
    StandardScaler(
    inputCol="features_numerical",outputCol="numerical_sc"
    )
])

In [0]:
pipeline_y = Pipeline(stages=[
    
        StringIndexer(
        inputCol = 'Stay_Days', outputCol = 'labelI'
    )
])
#A pipeline for y

I'll build the model

In [0]:
preprocessor = Pipeline(stages=[
    pipeline_categorical,pipeline_numerical, pipeline_y, 
    VectorAssembler(inputCols=["numerical_sc", "features_categorical"], outputCol="features_final"),
    
])

In [0]:
data_by_pipeline = preprocessor.fit(df_new_covid).transform(df_new_covid)
# Transforming of the  dataset by pipeline

In [0]:
covid_19 = data_by_pipeline.limit(250000)  # Sampling 250000 rows 

I'll split the data by 75 and 25

In [0]:
train_covid, test_covid = covid_19.randomSplit([0.75, 0.25], seed = 2023)
print("Training covid Dataset Count: " + str(train_covid.count()))
print("Test covid Dataset Count: " + str(test_covid.count()))

Training covid Dataset Count: 187068
Test covid Dataset Count: 62932


Logistic regression and OneVsRest Classifier

In [0]:
# Base classifier.
lr = LogisticRegression(maxIter=50, regParam=0.3, elasticNetParam=0.8, fitIntercept=True)

# One Vs Rest Classifier.
LR_OVR = OneVsRest(classifier=lr,labelCol="labelI", featuresCol="features_final")
# Train the model.
model_ovr = LR_OVR.fit(train_covid)

# score on test data.
y_pred = model_ovr.transform(test_covid)


In [0]:
#Calculate the accuracy like my dataset is balanced
evaluator = MulticlassClassificationEvaluator(
labelCol="labelI", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(y_pred)
print("Accuracy score is equal to  %g " % (accuracy))

Accuracy score is equal to  0.278316 


In [0]:
#confusion matrix
cm_dt_result = y_pred.crosstab("prediction", "labelI")
cm_dt_result = cm_dt_result.toPandas()
cm_dt_result.sort_values(by = ['prediction_labelI'])

,prediction_labelI,0.0,1.0,10.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,0.0,17515,14778,538,11375,7200,4173,2274,2120,1363,1032,564


In [0]:
y_true = y_pred.select(['labelI']).collect()
y_pred1 = y_pred.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred1))

              precision    recall  f1-score   support

         0.0       0.28      1.00      0.44     17515
         1.0       0.00      0.00      0.00     14778
         2.0       0.00      0.00      0.00     11375
         3.0       0.00      0.00      0.00      7200
         4.0       0.00      0.00      0.00      4173
         5.0       0.00      0.00      0.00      2274
         6.0       0.00      0.00      0.00      2120
         7.0       0.00      0.00      0.00      1363
         8.0       0.00      0.00      0.00      1032
         9.0       0.00      0.00      0.00       564
        10.0       0.00      0.00      0.00       538

    accuracy                           0.28     62932
   macro avg       0.03      0.09      0.04     62932
weighted avg       0.08      0.28      0.12     62932

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

DecisionTreeClassifier

In [0]:
dtc = DecisionTreeClassifier(labelCol="labelI", featuresCol="features_final", maxBins = 33)
dtc_Model = dtc.fit(train_covid)
dtc_Model_trasform = dtc_Model.transform(train_covid)
y_pred_dtc = dtc_Model.transform(test_covid)

In [0]:
evaluator = MulticlassClassificationEvaluator(
labelCol="labelI", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(y_pred_dtc)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.390167 


In [0]:
#confusion matrix
cm_dt_result_dtc = y_pred_dtc.crosstab("prediction", "labelI")
cm_dt_result_dtc = cm_dt_result_dtc.toPandas()
cm_dt_result_dtc.sort_values(by = ['prediction_labelI'])

,prediction_labelI,0.0,1.0,10.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,0.0,11569,6717,275,4820,1342,1921,1447,330,96,53,87
1,1.0,5364,6873,97,2113,506,1931,514,112,68,37,27
2,2.0,30,87,17,534,488,0,34,144,43,50,33
4,3.0,361,916,123,3778,4729,18,228,1341,604,820,331
5,4.0,174,182,11,90,48,303,34,11,6,3,5
3,7.0,17,3,15,40,87,0,17,182,546,69,81


In [0]:
y_true_dtc = y_pred_dtc.select(['labelI']).collect()
y_pred1_dtc = y_pred_dtc.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true_dtc, y_pred1_dtc))

              precision    recall  f1-score   support

         0.0       0.40      0.66      0.50     17515
         1.0       0.39      0.47      0.42     14778
         2.0       0.37      0.05      0.08     11375
         3.0       0.36      0.66      0.46      7200
         4.0       0.35      0.07      0.12      4173
         5.0       0.00      0.00      0.00      2274
         6.0       0.00      0.00      0.00      2120
         7.0       0.52      0.40      0.45      1363
         8.0       0.00      0.00      0.00      1032
         9.0       0.00      0.00      0.00       564
        10.0       0.00      0.00      0.00       538

    accuracy                           0.39     62932
   macro avg       0.22      0.21      0.19     62932
weighted avg       0.35      0.39      0.32     62932

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

In [0]:
from pyspark.ml.classification import LinearSVC

LinearSVC and OneVsRest

In [0]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)
# One Vs Rest Classifier.
LR_OVR = OneVsRest(classifier=lsvc,labelCol="labelI", featuresCol="features_final")
# Train the model.
model_ovr = LR_OVR.fit(train_covid)

# score on test data.
y_pred_lsvc = model_ovr.transform(test_covid)

In [0]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
labelCol="labelI", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(y_pred_lsvc)
print("Accuracy score is equal to  %g " % (accuracy))

Accuracy score is equal to  0.347852 


In [0]:
#confusion matrix
cm_dt_result_lsvc = y_pred_lsvc.crosstab("prediction", "labelI")
cm_dt_result_lsvc = cm_dt_result_lsvc.toPandas()
cm_dt_result_lsvc.sort_values(by = ['prediction_labelI'])

,prediction_labelI,0.0,1.0,10.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,0.0,14708,10782,320,5782,1623,2802,1681,329,126,72,95
3,1.0,522,1000,17,694,329,364,75,79,21,19,18
2,10.0,39,42,4,47,45,21,11,17,4,5,2
6,2.0,1389,1853,93,2883,2241,442,261,674,169,208,161
8,3.0,225,378,40,1263,1861,42,98,486,168,278,116
9,4.0,338,500,8,205,70,427,54,26,14,8,6
1,5.0,42,29,2,30,15,17,11,7,1,0,3
4,6.0,182,167,18,336,550,58,56,206,69,40,67
7,7.0,70,26,36,135,466,0,27,296,791,402,96
5,9.0,0,1,0,0,0,0,0,0,0,0,0


In [0]:
y_true_lsvc = y_pred_lsvc.select(['labelI']).collect()
y_pred1_lsvc = y_pred_lsvc.select(['prediction']).collect()

from sklearn.metrics import classification_report
print(classification_report(y_true_lsvc, y_pred1_lsvc))

              precision    recall  f1-score   support

         0.0       0.38      0.84      0.53     17515
         1.0       0.32      0.07      0.11     14778
         2.0       0.28      0.25      0.27     11375
         3.0       0.38      0.26      0.31      7200
         4.0       0.26      0.10      0.15      4173
         5.0       0.07      0.00      0.01      2274
         6.0       0.12      0.10      0.11      2120
         7.0       0.34      0.58      0.43      1363
         8.0       0.00      0.00      0.00      1032
         9.0       0.00      0.00      0.00       564
        10.0       0.02      0.01      0.01       538

    accuracy                           0.35     62932
   macro avg       0.20      0.20      0.17     62932
weighted avg       0.31      0.35      0.28     62932

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

Random Forest Classifier

In [0]:
rfc = RandomForestClassifier(featuresCol = 'features_final', labelCol = 'labelI', maxBins = 33)
rfc_Model = rfc.fit(train_covid)
rfc_Model_trasform = rfc_Model.transform(train_covid)
y_pred_rfc = rfc_Model.transform(test_covid)

In [0]:
evaluator = MulticlassClassificationEvaluator(
labelCol="labelI", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(y_pred_rfc)
print("Accuracy score is equal to  %g " % (accuracy))

Accuracy score is equal to  0.355987 


In [0]:
#confusion matrix
cm_dt_result_rfc = y_pred_rfc.crosstab("prediction", "labelI")
cm_dt_result_rfc = cm_dt_result_rfc.toPandas()
cm_dt_result_rfc.sort_values(by = ['prediction_labelI'])

,prediction_labelI,0.0,1.0,10.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,0.0,14517,9701,388,6964,3384,2714,1838,984,423,266,289
3,1.0,2726,4386,56,1594,610,1447,276,169,68,66,34
1,2.0,150,403,44,1774,1480,8,72,418,376,296,99
2,3.0,122,288,50,1043,1726,4,88,549,496,404,142


In [0]:
y_true_rfc = y_pred_rfc.select(['labelI']).collect()
y_pred1_rfc = y_pred_rfc.select(['prediction']).collect()

from sklearn.metrics import classification_report
print(classification_report(y_true_rfc, y_pred1_rfc))

              precision    recall  f1-score   support

         0.0       0.35      0.83      0.49     17515
         1.0       0.38      0.30      0.33     14778
         2.0       0.35      0.16      0.22     11375
         3.0       0.35      0.24      0.29      7200
         4.0       0.00      0.00      0.00      4173
         5.0       0.00      0.00      0.00      2274
         6.0       0.00      0.00      0.00      2120
         7.0       0.00      0.00      0.00      1363
         8.0       0.00      0.00      0.00      1032
         9.0       0.00      0.00      0.00       564
        10.0       0.00      0.00      0.00       538

    accuracy                           0.36     62932
   macro avg       0.13      0.14      0.12     62932
weighted avg       0.29      0.36      0.29     62932

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

Naive Bayes Classifier

In [0]:
from pyspark.ml.classification import NaiveBayes

nbm = NaiveBayes(smoothing=1.0, modelType="multinomial",labelCol="labelI", featuresCol="features_final")
nbm_Model = nbm.fit(train_covid)

y_pred_nbm = nbm_Model.transform(test_covid)

In [0]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
labelCol="labelI", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(y_pred_nbm)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.330468 


In [0]:
#confusion matrix
cm_dt_result_nbm = y_pred_nbm.crosstab("prediction", "labelI")
cm_dt_result_nbm = cm_dt_result_nbm.toPandas()
cm_dt_result_nbm.sort_values(by = ['prediction_labelI'])

,prediction_labelI,0.0,1.0,10.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,0.0,10759,6127,289,5173,2523,1639,1416,707,201,147,188
3,1.0,4916,6814,117,3694,1778,1847,505,408,156,200,74
2,10.0,4,2,0,1,3,0,3,0,0,0,0
4,2.0,339,396,29,524,521,114,83,194,101,103,50
6,3.0,1213,1170,65,1764,2123,395,216,644,491,450,189
8,4.0,227,229,10,121,64,163,31,16,2,5,5
1,5.0,30,14,6,21,12,12,6,3,2,1,2
5,7.0,27,26,22,77,175,3,14,147,408,126,56
7,8.0,0,0,0,0,1,0,0,1,2,0,0


In [0]:
y_true_nbm = y_pred_nbm.select(['labelI']).collect()
y_pred1_nbm = y_pred_nbm.select(['prediction']).collect()

from sklearn.metrics import classification_report
print(classification_report(y_true_nbm, y_pred1_nbm))

              precision    recall  f1-score   support

         0.0       0.37      0.61      0.46     17515
         1.0       0.33      0.46      0.39     14778
         2.0       0.21      0.05      0.08     11375
         3.0       0.24      0.29      0.27      7200
         4.0       0.19      0.04      0.06      4173
         5.0       0.06      0.00      0.01      2274
         6.0       0.00      0.00      0.00      2120
         7.0       0.38      0.30      0.33      1363
         8.0       0.00      0.00      0.00      1032
         9.0       0.00      0.00      0.00       564
        10.0       0.00      0.00      0.00       538

    accuracy                           0.33     62932
   macro avg       0.16      0.16      0.14     62932
weighted avg       0.27      0.33      0.27     62932

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

In [0]:
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit

param_grid = (
    ParamGridBuilder()
    .addGrid(rfc.maxDepth, [3,5,10,20])
    
    .build()
)

In [0]:
tune = TrainValidationSplit(estimator=rfc,
                           estimatorParamMaps=param_grid,
                           evaluator=MulticlassClassificationEvaluator(labelCol="labelI",metricName="accuracy"),
                           trainRatio=0.75,
                          collectSubModels=True)


model = tune.fit(train_covid)


In [0]:
predictions_y = model.transform(test_covid) # Using our best model 

In [0]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="labelI", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_y)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.418023 


The best model is Random Forest Classifier with an accuracy of 36%, very low, but the best, after the hyperparametre tuning our accuracy increases to 42%